In [ ]:


# Import necessary libraries
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import tkinter as tk
from tkinter import filedialog

import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

In [ ]:
# Set the base directory
base_dir = os.getcwd()  # Get the current working directory

# Construct the path to the dataset
dataset_dir = os.path.join(base_dir, 'dataset')

# Construct the path to the training directory
train_dir = os.path.join(dataset_dir, 'train')

# Construct the paths to each subdirectory for training
train_headtop_dir = os.path.join(train_dir, 'headtop')
train_helmet_dir = os.path.join(train_dir, 'helmet')
train_hoodie_dir = os.path.join(train_dir, 'hoodie')
train_no_headware_dir = os.path.join(train_dir, 'no_headwear')

# Print the total number of images in each training subdirectory
if os.path.exists(train_headtop_dir):
    print('Total training headtop images:', len(os.listdir(train_headtop_dir)))
else:
    print(f"Directory {train_headtop_dir} does not exist")

if os.path.exists(train_helmet_dir):
    print('Total training helmet images:', len(os.listdir(train_helmet_dir)))
else:
    print(f"Directory {train_helmet_dir} does not exist")

if os.path.exists(train_hoodie_dir):
    print('Total training hoodie images:', len(os.listdir(train_hoodie_dir)))
else:
    print(f"Directory {train_hoodie_dir} does not exist")

if os.path.exists(train_no_headware_dir):
    print('Total training no headware images:', len(os.listdir(train_no_headware_dir)))
else:
    print(f"Directory {train_no_headware_dir} does not exist")



In [ ]:
# Construct the path to the validation directory
val_dir = os.path.join(dataset_dir, 'val')

# Construct the paths to each subdirectory for validation
val_headtop_dir = os.path.join(val_dir, 'headtop')
val_helmet_dir = os.path.join(val_dir, 'helmet')
val_hoodie_dir = os.path.join(val_dir, 'hoodie')
val_no_headware_dir = os.path.join(val_dir, 'no_headwear')

# Print the total number of images in each validation subdirectory
if os.path.exists(val_headtop_dir):
    print('Total validation headtop images:', len(os.listdir(val_headtop_dir)))
else:
    print(f"Directory {val_headtop_dir} does not exist")

if os.path.exists(val_helmet_dir):
    print('Total validation helmet images:', len(os.listdir(val_helmet_dir)))
else:
    print(f"Directory {val_helmet_dir} does not exist")

if os.path.exists(val_hoodie_dir):
    print('Total validation hoodie images:', len(os.listdir(val_hoodie_dir)))
else:
    print(f"Directory {val_hoodie_dir} does not exist")

if os.path.exists(val_no_headware_dir):
    print('Total validation no headware images:', len(os.listdir(val_no_headware_dir)))
else:
    print(f"Directory {val_no_headware_dir} does not exist")

In [ ]:
# Create data generators
val_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Only rescale the images

validation_generator = val_datagen.flow_from_directory(
    val_dir,  # Use the validation directory
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical'  # Change to categorical for multi-class classification
)

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2),
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
# Define the simplified model with MaxPooling2D and Dropout
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(4, activation='softmax')
])
model.summary()

In [9]:
# Compile multi-class classification model
model.compile(loss='categorical_crossentropy', 
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['acc'])

In [ ]:
# Train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Model training
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=15,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stopping],
    verbose=1
)

In [11]:
# Save the model in the native Keras format
model_path = os.path.join(base_dir, 'model.keras')  # Construct the file path
model.save(model_path)  # Save the model

In [ ]:
# Set the base directory
script_dir = os.getcwd()  # Get the current working directory

# Load your pre-trained model
# Replace 'path/to/your/model.keras' with the actual path to your model file
model_path = os.path.join(script_dir, 'model.keras')
model = load_model(model_path)

# Create a Tkinter root window (it will be hidden)
root = tk.Tk()
root.withdraw()

# Open a file dialog and select files
file_paths = filedialog.askopenfilenames()

# Define the target size for the images (should match the input size of the model)
target_size = (300,300)

for path in file_paths:
    # Predicting images
    img = image.load_img(path, target_size=target_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    
    # Assuming the model outputs probabilities for each class
    class_index = np.argmax(classes[0])
    class_labels = ['Headtop', 'Helmet', 'Hoodie', 'No headwear']  
    print(f"{path} is a {class_labels[class_index]}")

In [ ]:
# Set the base directory
script_dir = os.getcwd()  # Get the current working directory

# Load your pre-trained model
# Replace 'path/to/your/model.keras' with the actual path to your model file
model_path = os.path.join(script_dir, 'model.keras')
model = load_model(model_path)

# Define class labels
class_labels = ['Headtop', 'Helmet', 'Hoodie', 'No headwear']

# Open the webcam
cap = cv2.VideoCapture(0)  # 0 is the default camera

# Define the target size for the images
target_size = (300, 300)

print("Press 'q' to exit the video feed.")

try:
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture video frame. Exiting...")
            break

        # Preprocess the frame
        resized_frame = cv2.resize(frame, target_size)  # Resize to match model input
        img_array = np.expand_dims(resized_frame, axis=0) / 255.0  # Normalize and add batch dimension

        # Make prediction
        predictions = model.predict(img_array)
        class_index = np.argmax(predictions[0])
        prediction_label = class_labels[class_index]
        confidence = predictions[0][class_index] * 100

        # Display the prediction on the frame
        cv2.putText(frame, f"{prediction_label} ({confidence:.2f}%)", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Show the frame
        cv2.imshow('Hat Detection', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("\nProgram interrupted by the user. Exiting...")

finally:
    # Release the webcam and close windows
    cap.release()
    cv2.destroyAllWindows()
    print("Resources released, video window closed.")
